# Part 1. From Zero to Snowflake in 50 Lines of Code

In this lab you will learn how to:

1. Create a session for Snowpark with Snowflake
2. Create a DB, Warehouse and Model Registry
3. Prep Data using the highly parallelisable vectorised UDTF functionality
4. Build/train a regression model with Snowpark ML
5. Register your model in the Model Registry
6. Deploy the model
7. Run the model

All this in 50 lines of code (less the library imports). Note - there are some TODOs along the way for you to update

## Prerequisites:
In a terminal please run:

conda env create -f conda_env.yml
 
conda activate snowpark-ml-hol

jupyter lab <---- this will load jupyter 

In [1]:
import json
import pandas as pd
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
from snowflake.snowpark.types import PandasDataFrameType, IntegerType, StringType, FloatType, DateType
from snowflake.ml.modeling.xgboost import XGBRegressor
from snowflake.ml.modeling.linear_model import LinearRegression
from snowflake.ml.registry import model_registry
from snowflake.ml._internal.utils import identifier

# 1.1 Reading Snowflake Connection Details, create a Session

TO DO: 

1. Create a JSON with your credentials and update the cell below

{
"account": "your_account_name", 
"user": "your_user_name",
"password": "insert_your_pwd_here",
"role": "ACCOUNTADMIN"
}

2. Update the location 

In [2]:
snowflake_connection_cfg = json.loads(open("/Users/mitaylor/Documents/creds/creds.json").read()) # <--- 2. Update here
session = Session.builder.configs(snowflake_connection_cfg).create()

# 1.2 Specify Your Database and Create a Virtual Warehouse

Snowflake seperates compute from storage, so we need a database AND a warehouse (compute environment) to run this stuff on.  Might as well create a model registry at the same time

In [3]:
session.sql("CREATE OR REPLACE DATABASE HOL_DEMO").collect()
session.sql("CREATE OR REPLACE WAREHOUSE ASYNC_WH WITH WAREHOUSE_SIZE='MEDIUM' WAREHOUSE_TYPE = 'SNOWPARK-OPTIMIZED'").collect()
REGISTRY_DATABASE_NAME = "MODEL_REGISTRY"
REGISTRY_SCHEMA_NAME = "PUBLIC"
model_registry.create_model_registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)
registry = model_registry.ModelRegistry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

create_model_registry() is in private preview since 0.2.0. Do not use it in production. 
/var/folders/97/8vc6xcbx4zd06p75xg9frdrw0000gn/T/ipykernel_5925/4250221149.py:6: DeprecationWarning: 
The `snowflake.ml.registry.model_registry.ModelRegistry` has been deprecated starting from version 1.2.0.
It will stay in the Private Preview phase. For future implementations, kindly utilize `snowflake.ml.registry.Registry`,
except when specifically required. The old model registry will be removed once all its primary functionalities are
fully integrated into the new registry.
        
  registry = model_registry.ModelRegistry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)


### EXTRA BIT, WHILE WE DECIDE ON DATA SHARES, PRE BUILT OR EVEN THIS CSV

In [4]:
session.write_pandas(pd.read_csv("test.csv"), table_name='FS_DATASET', auto_create_table=True, overwrite=True)

# 1.3 Get Your Data (Prepped)
In this case we're going to make a really simple lagging feature transformation for our time series dataset.  Nothign for you to do but run the cells, but note ANY pandas based manipulation could be performed here

In [5]:
sdf = session.table("FS_DATASET")
sdf = sdf.select(F.to_date(F.col('DATE')).as_('DATE'), "OPEN", "HIGH", "LOW", "CLOSE", "SYMBOL")

In [6]:
class ML_Prep:
    def end_partition(self, df):
        df.columns = ['_DATE', "_OPEN", "_HIGH", "_LOW", "_CLOSE", "_SYMBOL"]
        for i in range(1,6):
            df["_CLOSE-" + str(i)] = df["_CLOSE"].shift(i).bfill()
        yield df

ML_Prep.end_partition._sf_vectorized_input = pd.DataFrame

ml_prep_udtf = session.udtf.register(
    ML_Prep, # the class
    input_types=[PandasDataFrameType([DateType(), FloatType(), FloatType(), FloatType(), FloatType(), StringType()])], 
    output_schema=PandasDataFrameType([DateType(), FloatType(), FloatType(), FloatType(), FloatType(), StringType(),FloatType(),FloatType(),FloatType(),FloatType(),FloatType(),FloatType()],
                                      ['DATE', "OPEN", "HIGH", "LOW", "CLOSE", "SYMBOL", "CLOSE_M1", "CLOSE_M2", "CLOSE_M3", "CLOSE_M4", "CLOSE_M5"]),
    packages=["snowflake-snowpark-python", 'pandas'])  


In [7]:
sdf_prepped = sdf.select(ml_prep_udtf(*["DATE", "OPEN", "HIGH", "LOW", "CLOSE", "SYMBOL"]).over(partition_by=['SYMBOL']))
sdf_prepped.limit(10).to_pandas()

,DATE,OPEN,HIGH,LOW,CLOSE,SYMBOL,CLOSE_M1,CLOSE_M2,CLOSE_M3,CLOSE_M4,CLOSE_M5
0,2020-09-18,110.400002,110.879997,106.089996,106.839996,IBM,106.839996,106.839996,106.839996,106.839996,106.839996
1,2020-08-27,127.142502,127.485001,123.832497,125.010002,IBM,106.839996,106.839996,106.839996,106.839996,106.839996
2,2020-08-28,126.012497,126.442497,124.577499,124.807503,IBM,125.010002,106.839996,106.839996,106.839996,106.839996
3,2020-08-31,127.580002,131.000000,126.000000,129.039993,IBM,124.807503,125.010002,106.839996,106.839996,106.839996
4,2020-09-01,132.759995,134.800003,130.529999,134.179993,IBM,129.039993,124.807503,125.010002,106.839996,106.839996
5,2020-09-02,137.589996,137.979996,127.000000,131.399994,IBM,134.179993,129.039993,124.807503,125.010002,106.839996
6,2020-09-03,126.910004,128.839996,120.500000,120.879997,IBM,131.399994,134.179993,129.039993,124.807503,125.010002
7,2020-09-04,120.070000,123.699997,110.889999,120.959999,IBM,120.879997,131.399994,134.179993,129.039993,124.807503
8,2020-09-08,113.949997,118.989998,112.680000,112.820000,IBM,120.959999,120.879997,131.399994,134.179993,129.039993
9,2020-09-09,117.260002,119.139999,115.260002,117.320000,IBM,112.820000,120.959999,120.879997,131.399994,134.179993


In [8]:
sdf[['SYMBOL']].distinct().to_pandas()

,SYMBOL
0,IBM
1,AMZN
2,FDS
3,META


# 1.4.1 Choose Your Symbol, Train/Test Split and Model

We've got our data ready, but we need to make a few selections before we build our models

TO DO:
1. Choose the Symbol you want to build a model for
2. Pick the date range for your train/test split
3. Pick a regression model you want type

In [9]:
sdf_prepped_filt = sdf_prepped.filter((F.col("SYMBOL") == 'IBM'))
sdf_filt_train, sdf_filt_test = sdf_prepped_filt.filter((F.col("DATE") <= '2022-01-01')), sdf_prepped_filt.filter((F.col("DATE") > '2022-01-01'))
regressor = LinearRegression 

#sdf_prepped_filt = sdf_prepped.filter((F.col("SYMBOL") == "")) # <---- update 1.
#sdf_filt_train, sdf_filt_test = sdf_prepped_filt.filter((F.col("DATE") <= '')), sdf_prepped_filt.filter((F.col("DATE") > '')) # <---- update 2.
#regressor = # <---- update 3. hint one look at our imports cell

# 1.4.2 Train Your Model

Our model is almost ready to be trained, but we need to choose our inputs, targets, and outputs.  We could go off piste and alter model (hyper)parameters here too (https://docs.snowflake.com/en/developer-guide/snowpark-ml/reference/latest/api/modeling/snowflake.ml.modeling.linear_model.LinearRegression)

TO DO:
1. Select your input columns
2. Select your target(label) column
3. Choose your output column name

In [10]:
regressor = regressor(input_cols=["CLOSE_M1", "CLOSE_M2", "CLOSE_M3", "CLOSE_M4", "CLOSE_M5"],
                         label_cols=["CLOSE"],
                         output_cols=["CLOSE_PREDICT"])
regressor.fit(sdf_filt_train)

#regressor = regressor(input_cols=[], # <---- update 1.
#                         label_cols=[], # <---- update 2.
#                         output_cols=[]) # <---- update 3.
#regressor.fit(sdf_prepped_filt)

# 1.5 Register Your Model

Let's assume we love the first model, it's time to register it....

TO DO:
1. Choose a  model name
2. Choose a model version (note the combo of name and version needs to be unique)

In [12]:
MODEL_NAME = "REGRESSION_IBM"
MODEL_VERSION = "v1"
model = registry.log_model(model_name=MODEL_NAME,
                           model_version=MODEL_VERSION,
                           model=regressor,
                           tags={"stage": "testing", "classifier_type": "Lin_Reg"})

#MODEL_NAME = # <---- update 1.
#MODEL_VERSION = # <---- update 2.
#model = registry.log_model(model_name=MODEL_NAME,
#                           model_version=MODEL_VERSION,
#                           model=regressor,
#                           tags={"stage": "testing", "classifier_type": "xgb"},
#                           sample_input_data=sdf_prepped_filt.limit(10).to_pandas()[["CLOSE_M1", "CLOSE_M2", "CLOSE_M3", "CLOSE_M4", "CLOSE_M5"]],)

# 1.6 Deploy Your Model

Time to deploy the model...

In [13]:
model.deploy(deployment_name=MODEL_NAME + MODEL_VERSION,
             target_method="predict",
             permanent=True,
             options={"relax_version": True})

{'name': 'MODEL_REGISTRY.PUBLIC.model_predict_v7',
 'platform': <TargetPlatform.WAREHOUSE: 'warehouse'>,
 'target_method': 'predict',
 'signature': ModelSignature(
                     inputs=[
                         FeatureSpec(dtype=DataType.DOUBLE, name='CLOSE_M1'),
 		FeatureSpec(dtype=DataType.DOUBLE, name='CLOSE_M2'),
 		FeatureSpec(dtype=DataType.DOUBLE, name='CLOSE_M3'),
 		FeatureSpec(dtype=DataType.DOUBLE, name='CLOSE_M4'),
 		FeatureSpec(dtype=DataType.DOUBLE, name='CLOSE_M5')
                     ],
                     outputs=[
                         FeatureSpec(dtype=DataType.DOUBLE, name='CLOSE_M1'),
 		FeatureSpec(dtype=DataType.DOUBLE, name='CLOSE_M2'),
 		FeatureSpec(dtype=DataType.DOUBLE, name='CLOSE_M3'),
 		FeatureSpec(dtype=DataType.DOUBLE, name='CLOSE_M4'),
 		FeatureSpec(dtype=DataType.DOUBLE, name='CLOSE_M5'),
 		FeatureSpec(dtype=DataType.DOUBLE, name='CLOSE_PREDICT')
                     ]
                 ),
 'options': {'relax_version': True,
  'perman

# 1.7 Run Your Model

We're at the finish line!

In [15]:
model.predict(deployment_name=MODEL_NAME + MODEL_VERSION, data=sdf_filt_test).limit(20).to_pandas()

,DATE,OPEN,HIGH,LOW,CLOSE,SYMBOL,CLOSE_M1,CLOSE_M2,CLOSE_M3,CLOSE_M4,CLOSE_M5,CLOSE_PREDICT
0,2022-01-03,177.830002,182.880005,177.710007,182.009995,IBM,177.570007,178.199997,179.380005,179.289993,180.330002,174.761702
1,2022-01-04,182.630005,182.940002,179.119995,179.699997,IBM,182.009995,177.570007,178.199997,179.380005,179.289993,178.169860
2,2022-01-05,179.610001,180.169998,174.639999,174.919998,IBM,179.699997,182.009995,177.570007,178.199997,179.380005,177.002433
3,2022-01-06,172.699997,175.300003,171.639999,172.000000,IBM,174.919998,179.699997,182.009995,177.570007,178.199997,172.985956
4,2022-01-07,172.889999,174.139999,171.029999,172.169998,IBM,172.000000,174.919998,179.699997,182.009995,177.570007,169.914255
5,2022-09-19,149.309998,154.559998,149.100006,154.479996,IBM,126.849998,126.900002,127.099998,125.430000,127.309998,126.258712
6,2022-08-26,170.570007,171.050003,163.559998,163.619995,IBM,154.479996,126.849998,126.900002,127.099998,125.430000,148.152258
7,2022-08-29,161.149994,162.899994,159.820007,161.380005,IBM,163.619995,154.479996,126.849998,126.900002,127.099998,159.593992
8,2022-08-30,162.130005,162.559998,157.720001,158.910004,IBM,161.380005,163.619995,154.479996,126.849998,126.900002,159.867332
9,2022-08-31,160.309998,160.580002,157.139999,157.220001,IBM,158.910004,161.380005,163.619995,154.479996,126.849998,157.877329


## Makle sure you ruin this last line - it's needed for the next Part

In [16]:
model.predict(deployment_name=MODEL_NAME + MODEL_VERSION, data=sdf_filt_test).write.save_as_table("ML_PREDICT", mode="overwrite")